In [1]:
import sys
sys.path.insert(0, r'C:\Users\buzga\Desktop\School\Reaserch\Langone\ML_RA_EHR\DataModule')
sys.path.insert(0, r'C:\Users\buzga\Desktop\School\Reaserch\Langone\ML_RA_EHR')
sys.path.insert(0, r'C:\Users\buzga\Desktop\School\Reaserch\Langone\ML_RA_EHR\EnsambleModule')
import Data_Preparation
import EvaluationModule 
import ensamble_model 
import sklearn
from sklearn import datasets


In [2]:
path=r'C:\Users\buzga\Desktop\School\Reaserch\Langone\ML_RA_EHR\Dataset'
dataset = Data_Preparation.CoronnaCERTAINDataset(
    library_root=path,
    challenge="binary_classification", #option: regression, regression_delta, classification, binary_classification
    dataset='CORRONA CERTAIN', 
    process_approach='SC', #option: KVB, SC
    imputation="IterativeImputer", #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
    patient_group='bionaive TNF', #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
    drug_group='all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
    time_points=(0,3), 
    train_test_rate=0.8,
    remove_low_DAS = True,
    save_csv=True,
    random_state=2022)



train, train_loc = dataset.get_train()
test, test_loc = dataset.get_test()


## train test split 
X_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]
X_test = test.iloc[:,:-1]

y_test = test.iloc[:,-1]
len(y_test)+len(y_train)


feature engineering, drop columns due to 70% missing value: Index(['smkyrs', 'numcigs', 'rfstatus_impute', 'ccpstatus_impute',
       'statin_use'],
      dtype='object')
Missing values in train before imputation: 128
Missing values in train after imputation: 0
Missing values in test before imputation: 24
Missing values in test after imputation: 0
Label Encoder, 0: Non-responders (No Response), 1: Responders(Good, Moderate)
save file to: C:\Users\buzga\Desktop\School\Reaserch\Langone\ML_RA_EHR\Dataset\Coronna_Data_CERTAIN_binary_classification_SC_0M_3M_bionaive TNF_all\Train.csv
Label Encoder, 0: Non-responders (No Response), 1: Responders(Good, Moderate)
save file to: C:\Users\buzga\Desktop\School\Reaserch\Langone\ML_RA_EHR\Dataset\Coronna_Data_CERTAIN_binary_classification_SC_0M_3M_bionaive TNF_all\Test.csv


389

In [3]:
meta_learners=[ "Bayes", 'Logistic', "SVM", 'ANN','XGBoost','Random Forest', 'Tree']
from itertools import chain, combinations
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r+1) for r in range(len(s)))
estemators=[list(c) for c in list(powerset(meta_learners))]

In [4]:
aml=EvaluationModule.AutoBuild(seed=1,challenge='binary_classification')
parameters= {'ensamble_type':["Stacking"],'model_list':estemators, 'challange':['Classification'], 'meta_learner':meta_learners}
ensamble_model.test_hyper_parameters(parameters,train,test,dataset,aml)

  9%|▉         | 80/889 [09:53<2:16:08, 10.10s/it]